<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/main/TF_IDF_UP50%20con%20max%20y%20min.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cargar la base

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
base=pd.read_csv('/content/drive/MyDrive/base_formateada.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# Se incorporan las stopwords sugeridas por Federico
stopwords.extend(['señor', 'señora', 'mucha', 'gracia', 'año', 'pasado', 'cada', 'vez', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis' ,'siete', 'ocho', 'nueve', 'diez'])
stopwords.extend(['hoy', 'aquí', 'primer', 'lugar', 'primera', 'segunda', 'primero', 'segundo', 'siguiente', 'tercer'])
stopwords.extend(['convergencia', 'esquerra', 'republicana', 'grupo', 'parlamentaria', 'parlamentario', 'partido'])
stopwords.extend(['enmienda', 'votación', 'favor', 'abstención', 'ley', 'real', 'decreto', 'decretoley', 'resultado', 'voto', 'sé' ,'sí', 'silencio' ,'favor'])
stopwords.extend(['usted', 'señoría', 'presidente', 'presidenta', 'ministro', 'orden' , 'día', 'palabra', 'petición', 'posición', 'punto', 'vista', 'sesión', 'baldoví', 'duran', 'turno'])
stopwords.extend(['hacer', 'frente', 'puede' ,'ser', 'va', 'voy', 'decir'])
stopwords.extend(['millón', 'euro', 'emitido', 'efectuada', 'dio', 'comienzo', 'partido', 'queda', 'quedan', 'rechazada', 'aceptada', 'comienza', 'usted', 'sabe', 'abstención', 'diputado', 'gobierno'])
stopwords.extend(['continuación', 'votamos', 'telemático', 'republicanaizquierda', 'unidainiciativa', 'puede', 'bien', 'propuesta' ,'abstencion', 'mayoría', 'absoluta', 'pregunta', 'don', 'vamos', 'votar', 'llevar', 'cabo', 'millón', 'muchas', 'gracias'])
stopwords.extend(['podemosen', 'podemen', 'ministro', 'ministra', 'ss', 'ruego', 'aprobado', 'aprobada'])

In [5]:
# Se incorporan los nombres de los oradores como stopwords
from collections import Counter
nombres=[]
for i in base.namey.unique().tolist(): nombres.extend(str(i).split(' '))
nombres = [x.strip(' ') for x in nombres]
nombres = [x.strip(' †') for x in nombres]
nombres = [x.strip(',') for x in nombres]
nombres=list(Counter(nombres))
nombres.remove('')

In [6]:
stopwords.extend(nombres)

In [7]:
partidos=pd.read_csv('/content/drive/MyDrive/partidos.csv', encoding='latin1', sep=';')

In [8]:
stopwords.extend(partidos['nombre'].unique())
stopwords.extend(partidos['1'].unique())
stopwords.extend(partidos['2'].unique())
stopwords.extend(partidos['3'].unique())

In [9]:
len(stopwords)

2491

In [10]:
# elimino acentos de los tokens
a,b = 'áéèíóúü','aeeiouu'
trans = str.maketrans(a,b)
list = []
for row in base.values:
    row[20]=row[20].translate(trans)
    list.append(row)

base = pd.DataFrame(list, columns=base.columns)

In [11]:
# llevo las stopwords a minúsculas y elimino acentos
stopwords=[x.lower() for x in stopwords if type(x)==str]
stopwords=[x.translate(trans) for x in stopwords if type(x)==str]

# TF-IDF

In [12]:
# El hiperparámetro min_df del TF-IDF elimina las palabras con menor frecuencia. Por ejemplo, las palabras que aparecen en menos que el 0.001% de los discursos. 
# El hiperparámetro max_df del TF-IDF elimina las palabras con mayor frecuencia. Esto hace las veces de stopword específico. Por ejemplo, las palabras que aparecen en más que el que el 98% de los discursos. 
# En este ejercicio se eliminan estas palabras y luego se elimina el 50% con menor TF-IDF. 

from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(ngram_range=[2,2], stop_words=stopwords , max_df=0.98, min_df=0.00001)
bow = vect.fit_transform(base['tokens'])
total_features = len(vect.vocabulary_)

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['accion', 'ahuja', 'alcañiz', 'alderdi', 'alianza', 'alkartasuna', 'alternativa', 'amor', 'aragonesista', 'arreciado', 'asturias', 'barcena', 'barnuevo', 'bloque', 'canaria', 'canarias', 'canario', 'cantabria', 'capel', 'capitan', 'caralunya', 'cascos', 'castejon', 'catalunya', 'cataluña', 'centristas', 'centro', 'chunta', 'cienfuegos', 'ciudadana', 'ciudadania', 'ciudadanos', 'coalicion', 'compromis', 'comu', 'comun', 'comunista', 'congr', 'cristiana', 'democracia', 'democrata', 'democratas', 'democratica', 'democratico', 'democràtica', 'demòcrates', 'escandon', 'español', 'española', 'estefani', 'euskadi', 'euskadiko', 'euskal', 'eusko', 'euzko', 'existe', 'ezkerra', 'figueroa', 'foro', 'fortun', 'galego', 'galicia', 'geroa', 'herreria', 'hickman', 'huarte', 'independentista', 'iniciativa',

In [13]:
total_features

877129

In [15]:
sum_words = bow.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1])
vocabulary, _ = zip(*words_freq[:int(total_features * 0.5)])
less_vocabulary = vocabulary

In [16]:
voc=vect.vocabulary_.keys()-less_vocabulary

In [17]:
len(voc)

438565

In [18]:
cv=TfidfVectorizer(ngram_range=[2,2], vocabulary=voc , stop_words=stopwords)
vec = cv.fit(base['tokens'])

In [19]:
matrix = vec.transform(base['tokens'])

In [20]:
matrix.shape

(334421, 438565)

# Número óptimo de clusters

In [ ]:
# LDA en Sklearn tiene un resultado de score correspondiente al log-likelihood. En base a ese criterio se identifica el número óptimo de clusters. 

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

search_params = {'n_components': [20, 40, 60, 80]}
model = LatentDirichletAllocation(max_iter=500, learning_method='online', learning_offset=50.,random_state=0, cv=2)
gridsearch = GridSearchCV(model,
                          param_grid=search_params,
                          n_jobs=-1,
                          verbose=1)
gridsearch.fit(matrix)
best_lda = gridsearch.best_estimator_

In [ ]:
cv_results_df = pd.DataFrame(gridsearch.cv_results_)
cv_results_df

In [ ]:
import seaborn as sns
sns.set(rc={"figure.dpi":150, 'savefig.dpi':150})
sns.pointplot(x="param_n_components",
              y="mean_test_score",
              data=cv_results_df)

# LDA con 10 clusters

In [21]:
from sklearn.decomposition import LatentDirichletAllocation

In [22]:
# Acá hay un problema porque se sigue usando 10 clusters cuando en realidad habría que chequear el número óptimo de clusters una vez definida la cantidad de bigramas a utilizar. 
# Sin embargo, Text as data dice que el número de clústers es generalmente arbitrario, y recomienda arrancar probando con 10. 
lda = LatentDirichletAllocation(n_components=10, max_iter=5, learning_method='online', learning_offset=50.,random_state=0)     

In [23]:
lda.fit(matrix)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=5, random_state=0)

In [24]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx + 1))
        print(" , ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [25]:
display_topics(lda, feature_names=cv.get_feature_names_out(), no_top_words=20)

Topic 1:
vaya concluyendo , izquierda unida , feder izquierda , doña socialista , legarda uriart , acuerd celebracion , celebracion extraordinaria , tardà coma , beitialarrangoitia lizarrald , acepta alguna , suficient miembro , permanent pertenecient , mixto orama , capella farre , agirretxea urresti , mixto accensi , ciuro buldo , llamazar vasco , mixto quevedo , nueva formularla
Topic 2:
nombr socialista , mixto campuzano , diputada vicepresidenta , vicepresidenta adelant , campuzano canade , contestar interpelacion , vasco barandiaran , mixto roda , campuzano socialista , siento tiempo , reanudara mañana , interpelacion nombr , fijar popular , tiempo terminado , socialista simanca , diputada acabar , ciudadano girauta , exterior convenio , pleno reanudara , aitor vasco
Topic 3:
confeder unido , unido comu , vaya terminando , canaria mardon , momento guarden , seccion relativo , mañana hora , fijar nombr , nombr mixto , marea adelant , intervien mixto , jane popular , terminado tiem